In [145]:
from pulp import *
import pandas as pd
from typing import List, Tuple

In [146]:
alternatives = pd.read_csv("./data/games.csv")
criteria = pd.read_csv("./data/games_criteria.csv")

In [147]:
alternatives

,title,price,positive_reviews_percentage,number_of_reviews,system_requirements,content_volume,gameplay,audio,graphics,wishlist_position
0,Nebuchadnezzar,71.99,81,1085,3,5,8,6,6,1
1,Baba Is You,53.99,98,14974,1,8,7,3,4,2
2,ELDEN RING,249.00,92,494145,9,10,8,8,8,3
3,FTL Faster Than Light,35.99,95,52971,1,6,7,9,4,4
4,Superliminal,71.99,94,18197,4,4,6,3,5,5
5,Rogue Legacy 2,114.99,90,11809,3,6,6,5,6,6
6,Last Epoch,161.99,84,15629,7,9,9,6,7,7
7,Gears Tactics,142.99,75,5873,8,5,5,4,7,8
8,Path of Exile,0.00,87,197422,7,10,8,6,7,0
9,Divinity Original Sin 2,161.99,95,139068,7,9,9,9,9,0


In [148]:
criteria

,criterion_name,criterion_type,alpha,beta,weight
0,price,cost,NaN,0.0,7
1,positive_reviews_percentage,gain,0.0,100.0,10
2,number_of_reviews,gain,0.0,NaN,5
3,system_requirements,cost,10.0,1.0,6
4,content_volume,gain,1.0,10.0,6
5,gameplay,gain,1.0,10.0,8
6,audio,gain,1.0,10.0,3
7,graphics,gain,1.0,10.0,4
8,wishlist_position,cost,NaN,NaN,6


In [149]:
criteria["weight"] /= sum(criteria["weight"])
criteria

,criterion_name,criterion_type,alpha,beta,weight
0,price,cost,NaN,0.0,0.127273
1,positive_reviews_percentage,gain,0.0,100.0,0.181818
2,number_of_reviews,gain,0.0,NaN,0.090909
3,system_requirements,cost,10.0,1.0,0.109091
4,content_volume,gain,1.0,10.0,0.109091
5,gameplay,gain,1.0,10.0,0.145455
6,audio,gain,1.0,10.0,0.054545
7,graphics,gain,1.0,10.0,0.072727
8,wishlist_position,cost,NaN,NaN,0.109091


In [150]:

def fill_alpha_beta(alternatives: pd.DataFrame, criteria: pd.DataFrame) -> pd.DataFrame:
    """
    If any criteria don't have the values for the worst (alpha) or best (beta) performance,
    make them equal to the worst/best performances attained by the alternatives on this criterion
    """
    for idx, criterion in criteria.iterrows():
        if pd.isnull(criterion["alpha"]):
            if criterion["criterion_type"] == "gain":
                criterion["alpha"] = min(alternatives[criterion["criterion_name"]])
            if criterion["criterion_type"] == "cost":
                print(criterion["criterion_name"], max(alternatives[criterion["criterion_name"]]))
                criterion["alpha"] = max(alternatives[criterion["criterion_name"]])

        if pd.isnull(criterion["beta"]):
            if criterion["criterion_type"] == "gain":
                criterion["beta"] = max(alternatives[criterion["criterion_name"]])
            if criterion["criterion_type"] == "cost":
                criterion["beta"] = min(alternatives[criterion["criterion_name"]])
        criteria.loc[idx] = criterion

In [151]:
fill_alpha_beta(alternatives, criteria)
criteria

price 249.0
wishlist_position 8


,criterion_name,criterion_type,alpha,beta,weight
0,price,cost,249.0,0.0,0.127273
1,positive_reviews_percentage,gain,0.0,100.0,0.181818
2,number_of_reviews,gain,0.0,880572.0,0.090909
3,system_requirements,cost,10.0,1.0,0.109091
4,content_volume,gain,1.0,10.0,0.109091
5,gameplay,gain,1.0,10.0,0.145455
6,audio,gain,1.0,10.0,0.054545
7,graphics,gain,1.0,10.0,0.072727
8,wishlist_position,cost,8.0,0.0,0.109091



Expected preferences from project1:
- Path of Exile > Last Epoch
- Divinity Original Sin 2 > Gears Tactics
- Portal 2 > Superliminal
- Slay the Spire > Rogue Legacy 2

Additional comparisions:
- Terraria > Elden Ring
- Slay the Spire > Nebuchadnezzar
- Elden Ring > Nebuchadnezzar
- Nebuchadnezzar > Terraria (to introduce inconsistency)


In [152]:
# TODO: just to indicate what I intend to work on next
def ordinal_regression_model(alternatives: pd.DataFrame, criteria: pd.DataFrame, comparisions: List[Tuple[str, str, str]]) -> LpProblem:
    model = LpProblem(name="Ordinal Regression", sense=LpMinimize)
    